<a href="https://colab.research.google.com/github/mayanku1111/Speech-to-ASL/blob/main/Copying_of_FinalNUSproject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
!pip install -q kaggle
!pip install -q stanza
!pip install -q moviepy
import stanza
stanza.download('en')
nlp = stanza.Pipeline('en')
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d awsaf49/ljspeech-sr16k-dataset
import zipfile

with zipfile.ZipFile('/content/ljspeech-sr16k-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('ljspeech_sr16k_dataset')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 50.4 MB/s eta 0:00:00


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/awsaf49/ljspeech-sr16k-dataset
License(s): unknown
100% 2.17G/2.18G [00:24<00:00, 119MB/s]
100% 2.18G/2.18G [00:25<00:00, 93.0MB/s]


In [ ]:
# import pandas as pd
# df = pd.read_csv('/content/ljspeech_sr16k_dataset/metadata.csv')
# metadata_df = df[["id","sentence"]]
# metadata_df.head(5)

,id,sentence
0,LJ001-0001,"Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...
3,LJ001-0004,"produced the block books, which were the immed..."
4,LJ001-0005,the invention of movable metal letters in the ...


In [ ]:
# sentences_list = metadata_df['sentence'].tolist()[:60]


In [ ]:
# test_dataset = [
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0001.wav", sentences_list[0]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0002.wav", sentences_list[1]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0003.wav", sentences_list[2]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0004.wav", sentences_list[3]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0005.wav", sentences_list[4]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0006.wav", sentences_list[5]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0007.wav", sentences_list[6]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0008.wav", sentences_list[7]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0009.wav", sentences_list[8]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0010.wav", sentences_list[9]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0011.wav", sentences_list[10]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0012.wav", sentences_list[11]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0013.wav", sentences_list[12]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0014.wav", sentences_list[13]),
#     ("/content/ljspeech_sr16k_dataset/wavs/LJ001-0015.wav", sentences_list[14]),



# ]





In [ ]:
import tensorflow as tf
from keras import losses
from keras.saving import register_keras_serializable
import keras


frame_length = 256

frame_step = 160

fft_length = 384


@register_keras_serializable()
def CTCLoss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

from keras.models import load_model

model = load_model('/content/drive/MyDrive/my_Projmodel101.keras', custom_objects={'CTCLoss': CTCLoss})
model.summary()

Model: "DeepSpeech_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, None, 193)]       0         
                                                                 
 expand_dim (Reshape)        (None, None, 193, 1)      0         
                                                                 
 conv_1 (Conv2D)             (None, None, 97, 16)      3696      
                                                                 
 conv_1_bn (BatchNormalizat  (None, None, 97, 16)      64        
 ion)                                                            
                                                                 
 conv_1_relu (ReLU)          (None, None, 97, 16)      0         
                                                                 
 conv_2 (Conv2D)             (None, None, 49, 16)      59136     
                                                      

In [ ]:

from IPython.display import Javascript, display, Audio as IPythonAudio
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
!pip install -q pyspellchecker
from pydub import AudioSegment

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})

var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 19.3 MB/s eta 0:00:00


In [ ]:

audio = record(5)
audio.export("/content/output.wav", format="wav", parameters=["-b:a", "16"])

IPythonAudio("/content/output.wav", autoplay=True)


In [ ]:
!pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 18.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

characters = [x for x in "abcdefghijklmnopqrstuvwxyz'?! "]

char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token="")
num_to_char = keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True)
def decode_batch_predictions(pred, beam_width=50):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]


    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=False, beam_width=beam_width, top_paths=1)[0][0]

    output_text = []
    for result in results:
        result = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
        output_text.append(result)
    return output_text


def transcribe_single_voice(audio_file_path, beam_width=50):
    file = tf.io.read_file(audio_file_path)
    audio, _ = tf.audio.decode_wav(file)
    audio = tf.squeeze(audio, axis=-1)

    audio = tf.cast(audio, tf.float32)

    spectrogram = tf.signal.stft(
        audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length
    )
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)

    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)

    spectrogram = tf.expand_dims(spectrogram, 0)

    predictions = model.predict(spectrogram)

    transcription = decode_batch_predictions(predictions, beam_width=beam_width)[0]

    return transcription

custom_voice_path = "/content/ljspeech_sr16k_dataset/wavs/LJ001-0001.wav"
beam_width = 50

transcription = transcribe_single_voice(custom_voice_path, beam_width)

print("Transcription:", transcription)
from spellchecker import SpellChecker

spell = SpellChecker()


misspelled = transcription
corrected = " ".join(spell.correction(word) for word in misspelled.split())

print(corrected)

1/1 [==============================] - 5s 5s/step
Transcription: frtaing in the only sans whith which we arat presinti conserned difers from most ivfe not from al the arsic crafts represented in the exivision
freaking in the only sans with which we art present concerned divers from most if not from a the arsis crafts represented in the division


In [ ]:
# !pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 24.0 MB/s eta 0:00:00


In [ ]:
# predictions = []
# ground_truths = []

# for audio_path, true_transcription in test_dataset:
#     predicted_transcription = transcribe_single_voice(audio_path, beam_width)
#     predictions.append(predicted_transcription)
#     ground_truths.append(true_transcription)


1/1 [==============================] - 0s 74ms/step


In [ ]:
# import jiwer

# wer = jiwer.wer(ground_truths, predictions)
# print("Word Error Rate (WER):", wer)


Word Error Rate (WER): 0.4828897338403042


In [ ]:
# def word_level_accuracy(ground_truths, predictions):
#     total_words = 0
#     correct_words = 0

#     for true_transcription, pred_transcription in zip(ground_truths, predictions):
#         true_words = true_transcription.split()
#         pred_words = pred_transcription.split()

#         total_words += len(true_words)
#         correct_words += sum(tw == pw for tw, pw in zip(true_words, pred_words))

#     return correct_words / total_words if total_words > 0 else 0

# word_accuracy = word_level_accuracy(ground_truths, predictions)
# print("Word-level Accuracy:", word_accuracy)


Word-level Accuracy: 0.33079847908745247


In [ ]:

# Calculate Sentence-level Accuracy
# def sentence_level_accuracy(ground_truths, predictions):
#     correct_sentences = sum(gt == pred for gt, pred in zip(ground_truths, predictions))
#     return correct_sentences / len(ground_truths) if len(ground_truths) > 0 else 0

# sentence_accuracy = sentence_level_accuracy(ground_truths, predictions)
# print("Sentence-level Accuracy:", sentence_accuracy)

In [ ]:
import stanza

def parse(text):
    doc = nlp(text)
    result = []
    for sentence in doc.sentences:
        for word in sentence.words:
            result.append((word.text.lower(), word.lemma.lower()))
    return result
result = parse(corrected)
print(result)

resultant = []
for word in result:
  resultant.append(word[1])
import os
folder = os.getcwd()
filePrefix = folder + "/drive/MyDrive/bbc para1"


print(filePrefix)

files = [filePrefix +"/"+ word.lower() + "_.mp4" for word in resultant]
print(files)
existing_files = [file for file in files if os.path.exists(file)]
print(existing_files)

[('freaking', 'freak'), ('in', 'in'), ('the', 'the'), ('only', 'only'), ('sans', 'san'), ('with', 'with'), ('which', 'which'), ('we', 'we'), ('art', 'art'), ('present', 'present'), ('concerned', 'concern'), ('divers', 'diver'), ('from', 'from'), ('most', 'most'), ('if', 'if'), ('not', 'not'), ('from', 'from'), ('a', 'a'), ('the', 'the'), ('arsis', 'arsis'), ('crafts', 'craft'), ('represented', 'represent'), ('in', 'in'), ('the', 'the'), ('division', 'division')]
/content/drive/MyDrive/bbc para1
['/content/drive/MyDrive/bbc para1/freak_.mp4', '/content/drive/MyDrive/bbc para1/in_.mp4', '/content/drive/MyDrive/bbc para1/the_.mp4', '/content/drive/MyDrive/bbc para1/only_.mp4', '/content/drive/MyDrive/bbc para1/san_.mp4', '/content/drive/MyDrive/bbc para1/with_.mp4', '/content/drive/MyDrive/bbc para1/which_.mp4', '/content/drive/MyDrive/bbc para1/we_.mp4', '/content/drive/MyDrive/bbc para1/art_.mp4', '/content/drive/MyDrive/bbc para1/present_.mp4', '/content/drive/MyDrive/bbc para1/concern

In [ ]:

from IPython.display import HTML
from base64 import b64encode
from moviepy.editor import VideoFileClip, concatenate_videoclips
clips = [VideoFileClip(file) for file in existing_files]

final_clip = concatenate_videoclips(clips)

output_path = '/content/drive/MyDrive/output_video4.mp4'
final_clip.write_videofile(output_path)
from IPython.display import HTML
from base64 import b64encode

def display_video(path, autoplay=False):
    video_file = open(path, "rb").read()
    video_url = "data:video/mp4;base64," + b64encode(video_file).decode()
    autoplay_attr = "autoplay" if autoplay else ""
    return HTML(f"""<video width=600 controls {autoplay_attr}>
                      <source src="{video_url}" type="video/mp4">
                    </video>""")


Moviepy - Building video /content/drive/MyDrive/output_video4.mp4.
Moviepy - Writing video /content/drive/MyDrive/output_video4.mp4



t:   9%|▉         | 47/503 [00:00<00:06, 75.62it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/bbc para1/in_.mp4, 691200 bytes wanted but 0 bytes read,at frame 50/51, at time 1.67/1.67 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  19%|█▉        | 96/503 [00:02<00:12, 33.60it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/bbc para1/the_.mp4, 691200 bytes wanted but 0 bytes read,at frame 44/45, at time 1.47/1.50 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  40%|████      | 202/503 [00:05<00:09, 30.87it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /con

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/output_video4.mp4


In [ ]:
display_video(output_path,autoplay="True")

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.4 MB/s 

In [ ]:
import tensorflow as tf
from keras import losses
from keras.saving import register_keras_serializable
import keras
import numpy as np
from tensorflow import keras
from spellchecker import SpellChecker
import stanza
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips
import gradio as gr

# Define constants and load model
frame_length = 256
frame_step = 160
fft_length = 384

@register_keras_serializable()
def CTCLoss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

model = keras.models.load_model('/content/drive/MyDrive/my_Projmodel101.keras', custom_objects={'CTCLoss': CTCLoss})

characters = [x for x in "abcdefghijklmnopqrstuvwxyz'?! "]
char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token="")
num_to_char = keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True)

def decode_batch_predictions(pred, beam_width=50):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=False, beam_width=beam_width, top_paths=1)[0][0]
    output_text = []
    for result in results:
        result = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
        output_text.append(result)
    return output_text

def transcribe_single_voice(audio_file_path, beam_width=50):
    file = tf.io.read_file(audio_file_path)
    audio, _ = tf.audio.decode_wav(file)
    audio = tf.squeeze(audio, axis=-1)
    audio = tf.cast(audio, tf.float32)
    spectrogram = tf.signal.stft(audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)
    spectrogram = tf.expand_dims(spectrogram, 0)
    predictions = model.predict(spectrogram)
    transcription = decode_batch_predictions(predictions, beam_width=beam_width)[0]
    return transcription

# Initialize spell checker and stanza
spell = SpellChecker()
# stanza.download('en')
# nlp = stanza.Pipeline('en')

def spell_correct(text):
    corrected = " ".join(spell.correction(word) for word in text.split())
    return corrected

def parse(text):
    doc = nlp(text)
    result = []
    for sentence in doc.sentences:
        for word in sentence.words:
            result.append((word.text.lower(), word.lemma.lower()))
    return result

def generate_video(word_list):
    folder = os.getcwd()
    filePrefix = folder + "/drive/MyDrive/bbc para1"
    files = [filePrefix + "/" + word.lower() + "_.mp4" for word in word_list]
    existing_files = [file for file in files if os.path.exists(file)]

    if not existing_files:
        raise FileNotFoundError("No video clips found for the given words.")

    try:
        clips = [VideoFileClip(file) for file in existing_files]
        final_clip = concatenate_videoclips(clips)
        output_path = '/content/drive/MyDrive/output_video4.mp4'
        final_clip.write_videofile(output_path)
    except Exception as e:
        print(f"Error during video concatenation: {e}")
        return None

    return output_path

def process_audio(audio_file):
    try:
        transcription = transcribe_single_voice(audio_file)
        corrected = spell_correct(transcription)
        parsed = parse(corrected)
        word_list = [word[1] for word in parsed]
        video_path = generate_video(word_list)
        return video_path
    except Exception as e:
        print(f"Error processing audio file: {e}")
        return None

# Define Gradio interface
iface = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath"),
    outputs=gr.Video()
)

iface.launch(share=True,debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://21f6b40a5fec63451e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 3s 3s/step
Moviepy - Building video /content/drive/MyDrive/output_video4.mp4.
Moviepy - Writing video /content/drive/MyDrive/output_video4.mp4



t:   8%|▊         | 39/503 [00:00<00:02, 192.09it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/bbc para1/in_.mp4, 691200 bytes wanted but 0 bytes read,at frame 50/51, at time 1.67/1.67 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  18%|█▊        | 90/503 [00:00<00:02, 140.21it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/bbc para1/the_.mp4, 691200 bytes wanted but 0 bytes read,at frame 44/45, at time 1.47/1.50 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  39%|███▉      | 195/503 [00:01<00:02, 134.53it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/output_video4.mp4
